In [1]:
import spacy
import pandas as pd

import json
import spacy
import requests
import random
import time

import spacy
import tqdm
from collections import Counter

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

import pyLDAvis
import pyLDAvis.gensim

In [2]:
df = pd.read_csv("C:/Users/Rebeca OdC/Documents/A4/Social media analysis/obama_first_term.csv")
#corpus_text = df['lead_paragraph'].values

In [3]:
df['full_text']=df['headline']+df['lead_paragraph']
corpus_text = df['full_text'].values
df=df.dropna(subset=['full_text'])
df['lenght']=df['full_text'].apply(len)

In [4]:
#CONVERTING INTO LIST OF TOKEMS AND LEMATIZATION
nlp = spacy.load("en_core_web_sm")
corpus_text=df.full_text.values
documents_nlp = []
for d in tqdm.tqdm(nlp.pipe(corpus_text,n_process=5),total=len(corpus_text)):
    documents_nlp.append(d)
docs = [[token.lemma_.lower() for token in doc if  token.is_alpha] for doc in documents_nlp]

#To avoid 'or' and 'of', I remove words that have 2 characters:
docs = [[token for token in doc if len(token) > 2] for doc in docs]

#CREATING DICTIONNARY
id2word = Dictionary(docs)
id2word.filter_extremes(no_below=20, no_above=0.1)
corpus = [id2word.doc2bow(text) for text in docs]
#testing dictionnary
print(corpus[100])

100%|██████████| 1505/1505 [00:17<00:00, 85.53it/s] 


[(13, 1), (109, 1), (121, 1), (136, 1), (167, 2), (174, 1), (198, 1), (213, 1), (236, 1), (246, 1), (278, 1), (283, 1), (319, 1), (408, 1), (425, 1), (426, 1)]


In [5]:
#Finally, the model
lda_model = LdaModel(corpus=corpus,
                   id2word=id2word,
                   num_topics=15,
                   passes=20,
                   chunksize=2000)
from gensim.models.coherencemodel import CoherenceModel
cm = CoherenceModel(model=lda_model, corpus=corpus, coherence='u_mass')
cm.get_coherence()

#docs_2_topics will store the probability distribution of topics over documents.
docs_2_topics = lda_model[corpus]
docs_2_topics[24]

[(13, 0.9377758)]

In [6]:
label_dict = {i: ', '.join([token for token, score in lda_model.show_topic(i, topn=10)]) for i in range(0, lda_model.num_topics)}
for topic in label_dict:
    print (topic,':\t', label_dict[topic])

0 :	 into, political, ambassador, than, come, where, they, place, japan, find
1 :	 meeting, last, currency, week, issue, house, government, leader, summit, white
2 :	 when, they, two, minister, day, what, one, get, security, after
3 :	 government, arm, libya, thursday, jong, afghanistan, attack, kim, russia, taiwan
4 :	 war, europe, south, begin, european, question, take, editor, one, presidential
5 :	 company, one, myanmar, kong, most, big, make, foreign, become, chief
6 :	 military, official, asia, kong, warn, south, pakistan, sea, region, thursday
7 :	 clinton, secretary, state, hillary, rodham, she, her, policy, beijing, day
8 :	 nuclear, sanction, against, program, weapon, missile, test, iranian, would, monday
9 :	 news, state, human, diplomatic, release, right, report, call, run, political
10 :	 south, russia, talk, between, call, base, japan, visit, military, russian
11 :	 trade, they, their, other, buy, our, policy, can, should, america
12 :	 business, week, india, toward, smal

In [7]:
pyLDAvis.enable_notebook()
visualisation = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, sort_topics=False)
visualisation

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.113010  0.008030       1        1   5.348883
1     -0.014229 -0.036800       2        1   5.647906
2     -0.078198  0.022119       3        1   8.397815
3      0.015260  0.046634       4        1   5.581622
4     -0.005159  0.029640       5        1   5.192737
5     -0.090133 -0.054745       6        1   5.603528
6      0.226610  0.079672       7        1   6.328087
7      0.098789 -0.255033       8        1   5.971883
8      0.184480  0.150172       9        1   7.909497
9      0.043508 -0.141424      10        1   5.065587
10     0.124641 -0.004005      11        1   7.012694
11    -0.143532  0.047469      12        1  11.007059
12    -0.074179  0.000526      13        1   4.659989
13    -0.131357  0.065474      14        1   6.329134
14    -0.043491  0.042270      15        1   9.943577, topic_info=          Term        Freq       Total Category  logprob  loglift
175    nuclear  188.000000  188.000000  Default  30.0000  30.0000
52     clinton  141.000000  141.000000  Default  29.0000  29.0000
167   military  165.000000  165.000000  Default  28.0000  28.0000
87       state  164.000000  164.000000  Default  27.0000  27.0000
86   secretary  117.000000  117.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
36    security   16.179008  103.568251  Topic15  -5.0119   0.4517
164       make   16.226020  146.800852  Topic15  -5.0090   0.1058
19       first   15.971559  120.615943  Topic15  -5.0248   0.2864
211       take   15.794390  119.297105  Topic15  -5.0360   0.2863
218       they   15.813576  245.110149  Topic15  -5.0347  -0.4326

[702 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
283       3  0.055124  accord
283       7  0.385867  accord
283       9  0.110248  accord
283      11  0.110248  accord
283      13  0.082686  accord
...     ...       ...     ...
64        2  0.128335   young
64        4  0.128335   young
64        9  0.085556   young
64       11  0.042778   young
64       15  0.598895   young

[2951 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])

In [11]:
pyLDAvis.save_html(visualisation, 'obama_1_topic_modelling_visualisation.html')

In [9]:
topic_2_docs={}
for i, doc_2_topics in enumerate(docs_2_topics):
    for (topic,strength) in doc_2_topics:
        topic_2_docs.setdefault(topic,[]).append((i,strength))

In [10]:
#if we want to find the best article(s) for a topic
topic_number=1
best_doc = sorted(topic_2_docs[topic_number], key=lambda item: item[1], reverse=True)
print(label_dict[topic_number])
for doc in best_doc[:10]:
    print (corpus_text[doc[0]],doc[1])

meeting, last, currency, week, issue, house, government, leader, summit, white
Australia Drifts, With No Guiding StarIn easygoing Australia, the overwhelming focus on pedestrian domestic issues at election time is excusable. But the absence of genuine debate on foreign policy, trade or regional security has been alarming. With less than two weeks before election day, the Labor leader Julia Gillard, who became prime minister in June, and the conservative opposition leader, Tony Abbott, have set Australia on course for the foreign policy doldrums. 0.96666557
Taiwan’s Leader Faces Anger Over Storm ResponseTAIPEI, Taiwan  Flags are flying at half-staff during three days of national mourning to honor those killed by Typhoon Morakot two weeks ago. But anger, not sadness, remains the prevailing sentiment across Taiwan as President Ma Ying-jeou grapples with his worst political crisis since taking office last year. 0.95941836
China's Trade Surplus Rises to $27.1 Billion in OctoberBEIJING — A 